<div id="image">
<img src="https://www.imt-atlantique.fr/sites/default/files/logo_mt_0_0.png" WIDTH=280 HEIGHT=280>
</div>
<div id="subject">
<CENTER>
</br>
<font size="6"></br> JamBot Notebook</font></br></div>
</CENTER>
<CENTER>
<span style="color:blue">nicolas.herault@imt-atlantique.fr & nicolas.audepin@imt-atlantique.fr</span>
</CENTER>
</div>

**Introduction**

JamBot is a new model to generates a polyphonic music thanks to the traning of two LSTMs. First, we predicts a chord progression based on a chord embedding with a LSTM that we'll call chord_LSTM. A second one, that we'll call polyphonic_LSTM, generates polyphonic music from the predicted chord progression.

During the frist hours of our work on the project, we train the algorithm on The Beatles, and then generate music. From that, a pleasent melody came out, with some unpleasent notes. We decided to do two things :
On one hand, we want to train better the algorithm so that maybe the output want have those unpleasent notes. On the other hand, we want to filter the unpleasant notes on the output, so that maybe a nice melody will come out of this.

In this notebook, we want the reader to fully understand this problem, which can be quite complicated for someone who isn't a music expert. Therefore, we'll first make a short music lesson on the basic of music composition. Then, we'll explain with precision the dataset and the complex deep learning architecture of the algorithm. Finnaly, we'll show the results of our work on this algorithm. 

**A littre recall on music compostion**

An introduction to the basic of music compostion is essential to understand how the architecture is and why it has been choice to be like that. Here's a short but important recall on music compostion. 

*Bar :*
In musical notation, a bar or measure is a segment of time corresponding to a specific number of beats. Each beat corresponds to a note value. The boundaries between bars are indicated by vertical lines. In most, but not all music, a bar is 4 beats long.

*Equal Temperament :*
Almost all music uses a 12 tone equal temperamentsystem of tuning, in which the frequency interval between every pair of adjacent notes has the same ratio. Notes are: C,C♯/D♭,D,D♯/E♭,E,F,F♯/G♭,G,G♯/A♭,A,H, and then again C one octave higher. One cycle (e.g.,C to next C) is called an octave. Notes from different octaves are denoted with a number, for example D6 is the D from the sixth octave.

*Scale :*
A scale is a subset of (in most cases) 7 notes. Scales are defined by the pitch intervals between the notes of the scale. The most common scale is the major scale with the following pitch intervals: 2,2,1,2,2,2,1. The first note of the scale is called the root note. The pair of root note and scale is called a key. The major scale with the root note C contains the following notes: C2−→D2−→E1−→F2−→G2−→A2−→H1−→C.The natural minor scale has different pitch intervals than the major scale, but a natural minor scale with root note A contains exactly the same notes as a major scale with root note C. We call this a relative minor.

*Chords :*
A chord is a set of 3 or more notes played together. Chords are defined, like keys, by the pitch intervals and a starting note. The two most common types of chords are major chords and minor chords. We denote the major chords with the capital starting note, e.g.,F for an F major chord. For minor chords we add an m, e.g.,Dm for a D minor chord.

*Circle of Fifths :*
The circle of fifths, which is shown above, is the relationship among the 12 notes and their associated major and minor keys. It is a geometrical representation of the 12 notes that helps musicians switch between different keys and develop chord progressions. Choosing adjacent chords to form a chord progression often produces more harmonic sounding music.

**Settings**

A python function *settings.py* will be used by all the others functions. This function is there for tow things : first to settle the parameters of the music we want to generate. Then, it's also here to manage all the path that we need to use between the source and the saving folders. Here's below all the parameters that are in this function. Therefore, it is a summary of all the parameters of the algorithm. 

In [2]:
shifted = True

# Specifies the method how to add the chord information to the input vector
# 'embed' uses the chord embedding of the chord model
# 'onehot' encodes the chord as one hot vector
# 'int' just appends the chord id to the input vector
chord_embed_method = 'embed'

# Adds the count of the beat as a feature to the input vector
counter_feature = True
counter_size = 0
if counter_feature:
    counter_size = 3
    
# Appends also the next cord to the feature vector:
next_chord_feature = True

high_crop = 84   #84
low_crop = 24    #24
num_notes = 128
new_num_notes = high_crop - low_crop
chord_embedding_dim = 10

#double_sample_chords = False
double_sample_notes = True

sample_factor = 2

one_hot_input = False
collapse_octaves = True
discretize_time = False
offset_time = False
discritezition = 8
offset = 16

# Some parameters to extract the pianorolls
# fs = 4 for 8th notes
fs = 4
samples_per_bar = fs*2
octave = 12
melody_fs = 4


# Number of notes in extracted chords
chord_n = 3

# Number of notes in a key
key_n = 7

# Chord Vocabulary size
num_chords = 100

if shifted:
    num_chords = 50

**Dataset**

Now that we are aware of the basic of music composition, we can think of what can be the best dataset to represant this chords progression. The answer is MIDI files, which are a series of messages that indicated if there is at one instant a note, a change in the tempo, etc... ( you can see it like a coded pianoroll ) For our project, we download a MIDI files database on this website : https://colinraffel.com/projects/lmd/. It's about 100.000 songs, so pretty big enough to train our model. 


In [ ]:
# Let's initialize a source folder of MIDI files
source folder = []

This database of lots of MIDI files as to be shaped so that it can be used as a dataset. There for we'll applied a lot's of different functions on all those files. For that, we use the *data-processing.py* function. Here's below the main function of *data-processing.py* that summarize all the changing that are happening to each MIDI file. 

In [ ]:
def do_all_steps():
    print("source folder")
    print(source_folder)

    print('1 changing Tempo')
    change_tempo_folder(source_folder,tempo_folder1)
    # Commenter ce qui se passe 
    
    print('2 histogramming')
    save_histo_oct_from_midi_folder(tempo_folder1,histo_folder1)
    # Commenter ce qui se passe 

    print('3 make song histo')
    save_song_histo_from_histo(histo_folder1,song_histo_folder)
    # Commenter ce qui se passe 
    
    print('4 shifting midi files')
    shift_midi_files(song_histo_folder,tempo_folder1,tempo_folder2)
    # Commenter ce qui se passe 
    
    print('5 making note indexes')
    note_ind_folder(tempo_folder2,roll_folder)
    # Commenter ce qui se passe 

    print('6 histogramming')
    save_histo_oct_from_midi_folder(tempo_folder2,histo_folder2)
    # Commenter ce qui se passe 

    print('7 extracting chords')
    save_chords_from_histo(histo_folder2,chords_folder)
    # Commenter ce qui se passe 
    
    print('8 getting dictionary')
    chord_to_index, index_to_chord = make_chord_dict(chords_folder, num_chords)
    # Commenter ce qui se passe 
    
    print('9 converting chords to index sequences')
    save_index_from_chords(chords_folder,chords_index_folder)
    # Commenter ce qui se passe 

**Deep Learning Architecture**

<div id="image">
<img src="/homes/nherault/Bureau/Deep Learning 502/JamBot/JamBot_Architecure.png" >
</div>
<div id="subject">

**Results and improvements**

During our time working on JamBot, we succed to generates different music from different training and starting melody. The first one was the training on the *The Beatles* collection, which is about approximately 783 MIDI files, not all different music (there are many times the same music in different version). We then ask the algorithm to generates a music from the first 4 chords of "Hey Jude", from *The Beatles* too. We obtain a multitude of MIDI files from each different instrument that are registered in a dictionnary. Those aren't really important because after some listening, the melody is the same from each instruments. In those MIDI files, we can listen to a nice melody, which is unfortunetely wasted by some unpleasent high or low notes. Therefore, the idea of filtering some of those unpleasent notes is made so that we obtain a far better result. 
